https://data.cityofnewyork.us/dataset/Parks-Inspection-Program-Inspections/yg3y-7juh

## Park condition

If I have time I would like to incorporate this into the analysis.

Do areas with lower household income have more parks in poor conditions?

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/parks/Parks_Inspection_Program___Inspections.csv')

/Users/kylecaron/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,14,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
inspection_16 = df[(df['Inspection Year'] == 2016) & 
                  (df.InspectionType == 'PIP')].sort_values(by='Prop ID')
inspection_16;

In [4]:
inspection_16 = inspection_16[['Prop ID', 'Inspection ID', 'Season', 
                'Overall Condition', 'Cleanliness']]
inspection_16['Inspection_count'] = [1]*len(inspection_16)

In [5]:
len(inspection_16)

6000

In [6]:
inspection_16.isna().sum()

Prop ID              0
Inspection ID        0
Season               0
Overall Condition    0
Cleanliness          0
Inspection_count     0
dtype: int64

## Condition/Cleanliness not rated

In [7]:
(inspection_16['Overall Condition'] == 'N').sum()

29

In [8]:
(inspection_16['Cleanliness'] == 'N').sum()

29

In [9]:
((inspection_16['Overall Condition'] == 'N') & (inspection_16['Cleanliness'] =='N')).sum()

29

In [10]:
inspection_16 = inspection_16[inspection_16['Overall Condition'] != 'N']

# Strategy:

- Bool: if inspection in 2016 had unsatistfactory rating. 
- percentage of unacceptable conditions that year at that park
- number of inspections

In [11]:
def dummy(df, column, prefix_string):
    dummies = pd.get_dummies(inspection_16[column], prefix=prefix_string)
    return pd.concat((df, dummies), axis=1).drop(column, axis=1)

In [12]:
inspection_16 = dummy(inspection_16, 'Overall Condition', 'Condition')

In [13]:
inspection_16 = dummy(inspection_16, 'Cleanliness', 'Cleanliness')

## Groupby Prop ID

In [14]:
# check
len(inspection_16) == len(inspection_16['Inspection ID'].unique())

True

In [15]:
park_condition = inspection_16.groupby('Prop ID').sum().drop('Inspection ID', axis=1)
park_condition.head()

,Inspection_count,Condition_A,Condition_U,Cleanliness_A,Cleanliness_U
Prop ID,,,,,
B001,4,2,2,3,1
B002,3,2,1,2,1
B003,1,1,0,1,0
B006,1,1,0,1,0
B007,3,1,2,1,2


In [16]:
park_condition['Condition_rated_U'] = np.where(park_condition['Condition_U'] > 0, 1, 0)
park_condition['Cleanliness_rated_U'] = np.where(park_condition['Cleanliness_U'] > 0, 1, 0)

In [17]:
park_condition.drop(['Condition_A', 'Cleanliness_A'], axis=1, inplace=True)
park_condition.drop(['Condition_U', 'Cleanliness_U'], axis=1, inplace=True)

In [18]:
park_condition.head()

,Inspection_count,Condition_rated_U,Cleanliness_rated_U
Prop ID,,,
B001,4,1,1
B002,3,1,1
B003,1,0,0
B006,1,0,0
B007,3,1,1


In [19]:
park_condition.to_csv('../data/parks/park_condition_cleaned', index=True)

## Quick EDA

In [21]:
print('{}% of parks in poor condition at some point in 2016'.format(round(
    park_condition['Condition_rated_U'].sum()/len(park_condition)*100,2)))

print('{}% of parks considered unclean at some point in 2016'.format(round(
    park_condition['Cleanliness_rated_U'].sum()/len(park_condition)*100,2)))

21.37% of parks in poor condition at some point in 2016
12.65% of parks considered unclean at some point in 2016


What percentage of playgrounds were located in these?